Installing word2number library


In [ ]:
!pip install word2number

Importing things

In [ ]:
import re
from word2number import w2n  # Import the word2number library

Function to concat numbers like
**hello my email address is maliaas3 66@gmail.com**


In [ ]:
def concat_words_with_numbers(line):
    # Initialize a new list to store the result
    words = line.split()
    result = []
    # Initialize a variable to keep track of the index of the current word
    i = 0
    while i < len(words) - 1:  # Loop through the words list
        # Check if the last character of the current word and the first character of the next word are both digits
        if words[i][-1].isdigit() and words[i+1][0].isdigit():
            # Concatenate current and next word if condition is met
            result.append(words[i] + words[i+1])
            i += 2  # Skip the next word since it has been concatenated with the current one
        else:
            # Add the current word to the result if it does not meet the condition
            result.append(words[i])
            i += 1
    # Check if the last word was processed, if not, add it to the result
    if i == len(words) - 1:
        result.append(words[-1])
    return ' '.join(result)

In [ ]:
concat_words_with_numbers('hello my email address is maliaas3 66@gmail.com')

Extract Email address from a line ; It will help in parsing address having spaces


In [ ]:
def extract_email(text, domain_suffixes):
    # Find the starting index right after "is"
    if 'is' in text:
      start_index = text.find("is ") + 3
    elif 'address' in text :
      start_index = text.find("address ") + 8
    elif 'email' in text:
      start_index = text.find("email ") + 6
    else:
      start_index = 0
    # Find the ending index based on domain suffixes
    end_index = len(text)  # Default to the end of the text
    for suffix in domain_suffixes:
        temp_index = text.find(suffix)
        if temp_index != -1:
            end_index = min(end_index, temp_index + len(suffix))
            break  # Stop at the first match

    # Extract the raw email parts
    raw_email_parts = text[start_index:end_index]

    # Clean and concatenate the email parts
    email_address = raw_email_parts.replace(" ", "")

    return email_address

# Example text
text = "my email mali a bb a s 366@gmail.com"

# Call the function with domain suffixes to look for
email_address = extract_email(text, [".com", ".org"])

print(email_address)


Main Email Parser from transcript; It is running on examples I have seen in transcripts


In [ ]:
import re
from word2number import w2n

def convert_to_email_format(line):
    # Step to remove hyphens used in the email address format without affecting the numeric sequences
    line = re.sub(r'(?<=\w)-(?=\w)', '', line)
    # Normalize spacing and remove extra characters
    line = re.sub(r'\s+', ' ', line).replace(',', '')

    # Step 1: Convert number words to digits
    def word_to_num(match):
        word = match.group()
        try:
            return str(w2n.word_to_num(word))
        except ValueError:
            return word  # Return the original word if it's not a number word

    line = re.sub(r'\b(one|two|three|four|five|six|seven|eight|nine|zero)\b', word_to_num, line, flags=re.IGNORECASE)

    # Step 2: Handle "double" and "triple" for digits
    def handle_repetitions(match):
        rep_type, char = match.groups()
        count = 2 if rep_type == 'double' else 3
        if char.isdigit():
            return char * count
        else:
            # Convert word to number if needed and repeat
            try:
                number = w2n.word_to_num(char.lower())
                return str(number) * count
            except ValueError:
                return char * count

    # This pattern matches "double" or "triple" followed by either a digit or a word that can be a number
    line = re.sub(r'(double|triple) (\w+)', handle_repetitions, line, flags=re.IGNORECASE)

    # Step 3: Format email address
    line = re.sub(r'\b at \b', '@', line, flags=re.IGNORECASE)
    line = re.sub(r'\b dot \b', '.', line, flags=re.IGNORECASE)
    line = re.sub(r'\s*@\s*', '@', line)
    line = re.sub(r'\s*\.\s*', '.', line)

    # Condense multiple consecutive spaces into a single space globally
    line = re.sub(r'\s+', ' ', line)
    line = concat_words_with_numbers(line)
    line = extract_email(line, [".com", ".org"])
    return line

cases = ['hello my email address is maliaas3 66@gmail.com','my email address is mali, a double b a s three double six at gmail.com',
         'My email address is maliabbas3 double six at gmail dot com',
    'my email address is m-a-l-i-a-b-b-a-s-3-6 at gmail.com',
    'hello my email address is maliaas3 66 at gmail.com',
         'my email address is vines420 at gmail.com',
         ' my email address is malifourdouble2 at gmail.com',' my email address is vynz420 at gmid.com',
         'my email address is mali366 at gmail.com',
         ' my email address is mali366 at gmail.com']

for input_line in cases:
  output_line = convert_to_email_format(input_line)
  print("Input Line:", input_line)
  print("Output Line:", output_line)
